In [ ]:
from netgen.geom2d import SplineGeometry, unit_square
from ngsolve import *
from ngsolve.webgui import Draw
import ngsolve
from math import pi
geo = SplineGeometry()
geo.AddCircle(c=(0,0), r=1.0, bc="wall", maxh=0.1)
mesh = Mesh(geo.GenerateMesh())
mesh.Refine()
#mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))


# Analytische Geschwindigkeit (als GridFunction oder CoefficientFunction)
uexact = CoefficientFunction((
    sin(pi*x) * cos(pi*y),   # u(x,y)
    -cos(pi*x) * sin(pi*y)   # v(x,y)
))
pexact = sin(pi*x) * sin(pi*y)
# Laplace von uexact (Komponentenweise)
lapu_x = -pi**2 * sin(pi*x) * cos(pi*y) * 2
lapu_y = -pi**2 * (-cos(pi*x) * sin(pi*y)) * 2  # Minuszeichen kommt von v

# Gradient von p
dp_dx = pi * cos(pi*x) * sin(pi*y)
dp_dy = pi * sin(pi*x) * cos(pi*y)

# Gesamte rechte Seite f = -Δu + ∇p
f1 = CoefficientFunction((
    -lapu_x + dp_dx,
    -lapu_y + dp_dy
))

Draw(mesh)

order =3

# Taylor-Hood Spaces (P2 für u, P1 für p)
#V = VectorH1(mesh, order=order)  # Geschwindigkeit
V = VectorH1(mesh, order=order, dirichlet = "wall") # Geschwindigkeit
Q = H1(mesh, order = order-1)                        # Druck
M = NumberSpace(mesh) 
X = V*Q*M

# Funktionen definieren
(u, p ,l) = X.TrialFunction()
(v, q, n) = X.TestFunction()

nu = 1.0  # Viskosität
f = LinearForm(X)
f += f1*v*dx
#f +=  CoefficientFunction((0,x)) * v *dx
#f.Assemble()
# Bilinearform: schwache Formulierung
a = BilinearForm(X, symmetric=False)
a += nu * InnerProduct(grad(u), grad(v))*dx
a += div(v)*p*dx
a += div(u)*q*dx
a += (p*n + q*l) *dx

a.Assemble()
f.Assemble()

# Lösung
gfu = GridFunction(X)
gfu.components[0].Set(uexact, definedon=mesh.Boundaries("wall"))
Draw(gfu.components[0], mesh, "velocity")
f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(X.FreeDofs()) * f.vec
#solvers.BVP(bf=a, lf=f, gf=gfu)

#print("L2 error u:", sqrt(Integrate((gfu.components[0] - uexact)**2 , (gfu.components[1] - pexact)**2, mesh)))

# Visualisierung
(u_sol, p_sol,z) = gfu.components
print("L2 error u:", sqrt(Integrate((u_sol - uexact)**2 , mesh)))
print("L2 error p:", sqrt(Integrate((p_sol - pexact)**2 , mesh)))
Draw(u_sol, mesh, "velocity")
Draw(p_sol, mesh, "pressure")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

L2 error u: 7.086404303017299e-05
L2 error p: 1.9358488832160017


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:


#uexact = CoefficientFunction((    -r * (1 - r**2) * y, r * (1 - r**2) * x))

# Analytischer Druck
#pexact = (r + 3 - 15 * r**2) * y

In [6]:


r = sqrt(x**2 + y**2)
u1 = x * (1 - x**2-y**2)**2
u2 = y * (1 - x**2-y**2)**2
uexact = CoefficientFunction((u1, u2))

pexact = sin(x)*cos(y)

# Laplace-Anteil
lapu1 = 8*x**3 + 8*x *y**2 + (-16*x)*(1-x**2-y**2) 
lapu2 = 8*y**3 + 8*y *x**2 + (-16*y)*(1-x**2-y**2)


# Gradient von p
dpdx = cos(x) * cos(y)
dpdy = -sin(x) * sin(y)

f1 = CoefficientFunction((
    -lapu1 + dpdx,
    -lapu2 + dpdy
))

Draw(mesh)

order =4

# Taylor-Hood Spaces (P2 für u, P1 für p)
#V = VectorH1(mesh, order=order)  # Geschwindigkeit
V = VectorH1(mesh, order=order, dirichlet = "wall") # Geschwindigkeit
Q = H1(mesh, order = order-1)                        # Druck
M = NumberSpace(mesh) 
X = V*Q*M

# Funktionen definieren
(u, p ,l) = X.TrialFunction()
(v, q, n) = X.TestFunction()

nu = 1.0  # Viskosität
f = LinearForm(X)
f += f1*v*dx
#f +=  CoefficientFunction((0,x)) * v *dx
f.Assemble()
# Bilinearform: schwache Formulierung
a = BilinearForm(X, symmetric=False)
a += nu * InnerProduct(grad(u), grad(v))*dx
a += div(v)*p*dx
a += div(u)*q*dx
a += (p*n + q*l) *dx

a.Assemble()
f.Assemble()

# Lösung
gfu = GridFunction(X)
#gfu.components[0].Set(uexact, definedon=mesh.Boundaries("wall"))
Draw(gfu.components[0], mesh, "velocity")
#f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(X.FreeDofs()) * f.vec
#solvers.BVP(bf=a, lf=f, gf=gfu)

#print("L2 error u:", sqrt(Integrate((gfu.components[0] - uexact)**2 , (gfu.components[1] - pexact)**2, mesh)))

# Visualisierung
(u_sol, p_sol,z) = gfu.components
print("L2 error u:", sqrt(Integrate((u_sol - uexact)**2 , mesh)))
print("L2 error p:", sqrt(Integrate((p_sol - pexact)**2 , mesh)))
Draw(u_sol, mesh, "velocity")
Draw(p_sol, mesh, "pressure")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

L2 error u: 0.3236043187581708
L2 error p: 1.9815113269551106


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:

r = sqrt(x**2 + y**2)
u1 = -4*y * (1 - x**2 - y**2)
u2 = 4*x*(1 - x**2 - y**2)
uexact = CoefficientFunction((u1, u2))

pexact = sin(x)*cos(y)

# Laplace-Anteil
lapu1 = 8*y +24 * y
lapu2 = -8*x -24*x


# Gradient von p
dpdx = cos(x) * cos(y)
dpdy = -sin(x) * sin(y)

f1 = CoefficientFunction((
    -lapu1 + dpdx,
    -lapu2 + dpdy
))
#f1 = CF((0,x))
Draw(mesh)

order =3

# Taylor-Hood Spaces (P2 für u, P1 für p)
#V = VectorH1(mesh, order=order)  # Geschwindigkeit
V = VectorH1(mesh, order=order, dirichlet = "wall") # Geschwindigkeit
Q = H1(mesh, order = order-1)                        # Druck
M = NumberSpace(mesh) 
X = V*Q*M

# Funktionen definieren
(u, p ,l) = X.TrialFunction()
(v, q, n) = X.TestFunction()

nu = 1.0  # Viskosität
f = LinearForm(X)
f += f1*v*dx
#f +=  CoefficientFunction((0,x)) * v *dx
f.Assemble()
# Bilinearform: schwache Formulierung
a = BilinearForm(X, symmetric=False)
a += nu * InnerProduct(grad(u), grad(v))*dx
a += div(v)*p*dx
a += div(u)*q*dx
a += (p*n + q*l) *dx

a.Assemble()
f.Assemble()

# Lösung
gfu = GridFunction(X)
#gfu.components[0].Set(uexact, definedon=mesh.Boundaries("wall"))
Draw(gfu.components[0], mesh, "velocity")
#f.vec.data -= a.mat * gfu.vec
gfu.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec
#solvers.BVP(bf=a, lf=f, gf=gfu)

#print("L2 error u:", sqrt(Integrate((gfu.components[0] - uexact)**2 , (gfu.components[1] - pexact)**2, mesh)))

# Visualisierung
(u_sol, p_sol,z) = gfu.components
print("L2 error u:", sqrt(Integrate((u_sol - uexact)**2 , mesh)))
print("L2 error p:", sqrt(Integrate((p_sol - pexact)**2 , mesh)))
Draw(u_sol, mesh, "velocity")
Draw(p_sol, mesh, "pressure")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

L2 error u: 2.014704055721146
L2 error p: 0.7716180386969146


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene